# Setup

In [ ]:
#!/bin/python3 -m pip install evaluate nltk nibabel pydicom matplotlib pillow med2image rouge_score jax flax petl

# LongT5

In [ ]:

train_file = "train.csv"
eval_file = "val.csv"
test_file = "test.csv"

gradient_checkpointing = True  # would allow a higher batch size, because it would use less memory, but would slow down training

#model_name = "google/long-t5-tglobal-large"
#model_name = "google-long-t5-tglobal-large"
model_name = "google/long-t5-tglobal-base"
model_name_safe = model_name.replace("/", "-")
output_folder = f"output_{model_name_safe}"

!nohup /bin/python3 run_summarization_flax.py --output_dir {output_folder} --model_name_or_path {model_name} --tokenizer_name {model_name} --train_file {train_file} --validation_file={eval_file} --test_file={test_file} --do_train --do_eval --do_predict --predict_with_generate --num_train_epochs=16 --learning_rate=6e-5 --warmup_steps=100 --per_device_train_batch_size=1 --per_device_eval_batch_size=2 --overwrite_output_dir --max_source_length=3600 --max_target_length=500 --gradient_checkpointing={gradient_checkpointing} --save_steps=2000 --logging_steps=1000 --eval_steps=10000 --seed=42 --weight_decay=0.05 --overwrite_cache=True --per_device_gradient_accumulation_steps=2 --dtype="bfloat16" #--multisteps=True #--padding="longest"
#

# other parameters to try
# gradient_checkpointing = True  
# weight_decay = 0.01

In [ ]:
'''
load huggingface pytorch model
convert it to flax
save the flax huggingface model
'''

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import flax
import flax.linen as nn
import flax.jax_utils as flax_utils

# load the huggingface model
tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/long-t5-tglobal-base")

# convert the huggingface model to flax
flax_model = flax_utils.pure_fn(model.to_flax())
flax_model = flax.jax_utils.replicate(flax_model)

# save the flax huggingface model
flax.savez_compressed("long-t5-tglobal-xl.npz", flax_model.params)





In [ ]:
'''
get all lines in nb.log that start with "Epoch..."
'''

import re

with open("nb.log", "r") as f:
    lines = f.readlines()
    for line in lines:
        if re.match("^Epoch", line):
            print(line, end="")
